In [1]:
!pip install recordlinkage --quiet
!pip install sklearn --quiet

In [2]:
import numpy as np 
import pandas as pd
import sklearn
import recordlinkage as rl
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [3]:
companies = pd.read_csv("alignedSchemas/companiesAligned.csv")
companies.head(10)

,Unnamed: 0,name,headquarter,employees,industry,website,ticker,ceo,revenue_M,marketcap_M,shareprice
0,0,GROUPON,"600 W CHICAGO AVE SUITE 400 CHICAGO, IL 60616","1,001 TO 5,000",INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN,NaN
1,1,E-TECHNOLOGIES,AUCKLAND,2 TO 10,INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN,NaN
2,2,YAKSHNA SOLUTIONS,"HERNDON, VA",11 TO 50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,THRU TUBING SOLUTIONS,OKLAHOMA CITY,51 TO 200,"ENERGY, MINING & UTILITIES",NaN,NaN,NaN,NaN,NaN,NaN
4,4,CLEVELAND CLIFFS INC,"CLEVELAND, OH","5,001 TO 10,000",PERSONAL CONSUMER SERVICES,NaN,NaN,NaN,NaN,NaN,NaN
5,5,SISU,LOS ANGELES,11 TO 50,RESTAURANTS & CAFES,NaN,NaN,NaN,NaN,NaN,NaN
6,6,TSMC,"SAN JOSE, CA",51 TO 200,MANUFACTURING,NaN,NaN,NaN,NaN,NaN,NaN
7,7,MATHIS BROTHERS FURNITURE,"OKLAHOMA CITY, OKLAHOMA","1,001 TO 5,000",RETAIL & WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN
8,8,IVYTECH SOLUTIONS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,VERTIV,"1050 DEARBORN DRIVE, COLUMBUS, OH 43085, UNITE...","10,000+",TELECOMMUNICATIONS,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
companies.describe()

,Unnamed: 0,revenue_M,shareprice
count,64608.000000,28709.000000,10657.000000
mean,5375.459695,171.896685,191.114484
std,5217.961322,231.459789,6260.444685
min,0.000000,0.000000,0.010000
25%,1138.000000,7.464000,8.710000
50%,3741.000000,70.000000,26.710000
75%,8064.250000,241.000000,66.870000
max,20461.000000,2933.000000,444654.000000


In [5]:
indexer = rl.Index()
indexer.block('name')
pairs = indexer.index(companies, companies)

print(pairs)

MultiIndex([(    0,     0),
            (    0, 20812),
            (    0, 44061),
            (    0, 62588),
            (20812,     0),
            (20812, 20812),
            (20812, 44061),
            (20812, 62588),
            (44061,     0),
            (44061, 20812),
            ...
            (64593, 64593),
            (64594, 64594),
            (64595, 64595),
            (64596, 64596),
            (64598, 64598),
            (64601, 64601),
            (64602, 64602),
            (64603, 64603),
            (64606, 64606),
            (64607, 64607)],
           length=178227)


In [6]:
compare = rl.Compare()

compare.exact('name', 'name', label="name")
compare.exact('employees', 'employees', label='employees')
compare.string('website', 'website', method='jarowinkler', label="website")
compare.string('ticker', 'ticker', method='jarowinkler', label="ticker")
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo")

features = compare.compute(pairs, companies, companies)

In [7]:
matches = features[features.sum(axis=1) > 2].reset_index()
matches['score'] = matches.loc[:, 'name':'ceo'].sum(axis=1)
matches.head(25)
toDrop = ['name', 'employees', 'website', 'ticker', 'ceo', 'score']
matches = matches.drop(toDrop, axis=1)
matches = pd.MultiIndex.from_frame(matches)

In [8]:
golden_pairs = companies[0:5000]

In [10]:
# Initialize the classifier
logreg = rl.LogisticRegressionClassifier()

# Train the classifier
logreg.fit(golden_pairs, matches)
print ("Intercept: ", logreg.intercept)
print ("Coefficients: ", logreg.coefficients)

ValueError: could not convert string to float: 'GROUPON'